# Day 11: Plutonian Pebbles

## Part One

The ancient civilization on Pluto was known for its ability to manipulate spacetime, and while The Historians explore their infinite corridors, you've noticed a strange set of physics-defying stones.

At first glance, they seem like normal stones: they're arranged in a perfectly straight line, and each stone has a number engraved on it.

The strange part is that every time you blink, the stones change.

Sometimes, the number engraved on a stone changes. Other times, a stone might split in two, causing all the other stones to shift over a bit to make room in their perfectly straight line.

As you observe them for a while, you find that the stones have a consistent behavior. Every time you blink, the stones each simultaneously change according to the first applicable rule in this list:

* If the stone is engraved with the number 0, it is replaced by a stone engraved with the number 1.
* If the stone is engraved with a number that has an even number of digits, it is replaced by two stones. The left half of the digits are engraved on the new left stone, and the right half of the digits are engraved on the new right stone. (The new numbers don't keep extra leading zeroes: 1000 would become stones 10 and 0.)
* If none of the other rules apply, the stone is replaced by a new stone; the old stone's number multiplied by 2024 is engraved on the new stone.

No matter how the stones change, their order is preserved, and they stay on their perfectly straight line.

How will the stones evolve if you keep blinking at them? You take a note of the number engraved on each stone in the line (your puzzle input).

If you have an arrangement of five stones engraved with the numbers `0 1 10 99 999` and you blink once, the stones transform as follows:

* The first stone, 0, becomes a stone marked 1.
* The second stone, 1, is multiplied by 2024 to become 2024.
* The third stone, 10, is split into a stone marked 1 followed by a stone marked 0.
* The fourth stone, 99, is split into two stones marked 9.
* The fifth stone, 999, is replaced by a stone marked 2021976.

So, after blinking once, your five stones would become an arrangement of seven stones engraved with the numbers `1 2024 1 0 9 9 2021976`.

Here is a longer example:

Initial arrangement:

`125 17`

After 1 blink:

`253000 1 7`

After 2 blinks:

`253 0 2024 14168`

After 3 blinks:

`512072 1 20 24 28676032`

After 4 blinks:

`512 72 2024 2 0 2 4 2867 6032`

After 5 blinks:

`1036288 7 2 20 24 4048 1 4048 8096 28 67 60 32`

After 6 blinks:

`2097446912 14168 4048 2 0 2 4 40 48 2024 40 48 80 96 2 8 6 7 6 0 3 2`

In this example, after blinking six times, you would have `22` stones. After blinking 25 times, you would have `55312` stones!

Consider the arrangement of stones in front of you. How many stones will you have after blinking 25 times?

In [1]:
const input = true ?
	`0 7 198844 5687836 58 2478 25475 894` :
	`125 17`

type RuleFunction = (stone: string) => (string | string[] | false)

const rules: RuleFunction[] = [
	(stone) => stone == '0' ? '1' : false,
	(stone) => stone.length % 2 == 0 ?
		[
			''.concat(parseInt(stone.slice(0, stone.length / 2), 10)),
			''.concat(parseInt(stone.slice(stone.length / 2), 10))
		] :
		false,
	(stone) => ''.concat(parseInt(stone, 10) * 2024),
]

In [2]:
const stones = input.split(' ')

function blink(subjects:string[]): string[] {
	return subjects.reduce((line:string[], stone:string) => {
		for (let i = 0; i < rules.length; i++) {
			let output = rules[i](stone)
			if (output !== false) {
				line = [...line, output]
				break
			}
		}
		return line
	}, []).flat()
}

let partOne:string[] = [...stones]
for (let i = 0; i < 25; i++) {
	partOne = blink(partOne)
}
console.log(`Part one stone line count: ${partOne.length}`)

Part one stone line count: 216996


## Part One Thoughts

Splitting it into an array and continuously concatting it (while flatting) is super in-effecient, but it does work. It took `3m6s` on a *Intel(R) Xeon(R) CPU E3-1220 V2 @ 3.10GHz* single thread) on my input which is the longest running one of these so far, but it produced the correct answer. Would probably be best to operate on the string itself instead of on iteration of arrays.

## Part Two

The Historians sure are taking a long time. To be fair, the infinite corridors are very large.

How many stones would you have after blinking a total of `75` times?

## Part Two Plan

Clearly, this part is just testing the efficiency of part one.
Since the stones have no impact on each other, I will iterate on each stone independently
in a tree like approach when it needs to split, finally, I'll sum up the results at the end.

I'll also update the split rule to be a string so I can destructure the results for detecting a split.

In [3]:
rules[1] = (stone:string) => {
	return stone.length % 2 == 0 ?
	[
		parseInt(stone.slice(0, stone.length / 2), 10),
		parseInt(stone.slice(stone.length / 2), 10)
	].join(' ') :
	false
}
const cache: Map<string,int> = new Map()
function blink(stone:string, iteration:int = 75): int {
	if (iteration == 0) {
		return 1
	}
	let cacheKey = `${stone},${iteration}`
	if (cache.has(cacheKey)) {
		return cache.get(cacheKey)
	}
	for (let i = 0; i <= rules.length; i++) {
		let output = rules[i](stone)
		if (output !== false) {
			let [left, right] = output.split(' ')
			if (right) {
				let sum = blink(left, iteration - 1) + blink(right, iteration - 1)
				cache.set(cacheKey, sum)
				return sum
			}
			return blink(left, iteration - 1)
		}
	}
}
let partTwoCount = input.split(' ').reduce((sum:int, stone: string) => {
	return sum + blink(stone)
}, 0)

console.log(`Part two stone line count: ${partTwoCount}`)

Part two stone line count: 257335372288947
